# Lab 3

### Team Members:
 - Alex Lopez
 - Chris Haub
 - Erin McClure-Price
 - Chad Kwong

#### Library Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import graphlab as gl
import warnings
#import turicreate as tc


#### Data Import

In [2]:
%%time

#Loading all data file separately 
vle = pd.read_csv('vle.csv')
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
studentAssessments = pd.read_csv('studentAssessment.csv')
studentInfo = pd.read_csv('studentInfo.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')
studentVle = pd.read_csv('studentVle.csv')

CPU times: total: 2.55 s
Wall time: 3.87 s


In [55]:
%%time
dfFull = pd.merge(studentVle, studentInfo, on=['code_module', 'code_presentation', 'id_student'])
print(dfFull.shape)
dfFull.head()

(10655280, 15)
Wall time: 4.86 s


,code_module,code_presentation,id_student,id_site,date,sum_click,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,28400,546652,-10,4,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
1,AAA,2013J,28400,546652,-10,1,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,28400,546652,-10,1,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
3,AAA,2013J,28400,546614,-10,11,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
4,AAA,2013J,28400,546714,-10,1,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass


# Business Understanding 1 #
*Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?*

# Data Understanding 1 #
*Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?*

 - VLE stands for Virtual Learning Environment
 - VLE table records interactions with VLE 
 - 32,593 students
 - 22 courses
 

In [3]:
%%time
# Randomly sampling 4,000 records StudentVle Table with Million Rows.
studentVleSample = studentVle.sample(n=4000, random_state = 777)
print(studentVleSample.shape)

# vle
# assessments
# courses
# studentAssessments
# studentInfo
# studentRegistration
# studentVleSample

df = pd.merge(studentVleSample, studentInfo, on=['code_module', 'code_presentation', 'id_student'])
df2 = pd.merge(studentAssessments, assessments, on='id_assessment')
df = pd.merge(df, df2, on=['code_module', 'code_presentation', 'id_student'])
print(df.shape)
df.head()

(4000, 12)
(10744851, 28)
CPU times: total: 10.9 s
Wall time: 34.8 s


,code_module,code_presentation,id_student,id_site,date_x,sum_click,gender,region,highest_education,imd_band,...,studied_credits,disability,final_result,id_assessment,date_submitted,is_banked,score,assessment_type,date_y,weight
0,BBB,2014J,584539,913685,17,1,F,Scotland,HE Qualification,10-20,...,60,N,Pass,15020,17,0,100.0,TMA,19.0,0.0
1,BBB,2014J,584539,913685,17,1,F,Scotland,HE Qualification,10-20,...,60,N,Pass,15021,51,0,70.0,TMA,54.0,10.0
2,BBB,2014J,584539,913685,17,1,F,Scotland,HE Qualification,10-20,...,60,N,Pass,15022,107,0,72.0,TMA,110.0,20.0
3,BBB,2014J,584539,913685,17,1,F,Scotland,HE Qualification,10-20,...,60,N,Pass,15023,152,0,60.0,TMA,152.0,35.0
4,BBB,2014J,584539,913685,17,1,F,Scotland,HE Qualification,10-20,...,60,N,Pass,15024,206,0,63.0,TMA,201.0,35.0


In [6]:
df.dtypes

code_module              object
code_presentation        object
id_student                int64
id_site                   int64
date_x                    int64
sum_click                 int64
gender                   object
region                   object
highest_education        object
imd_band                 object
age_band                 object
num_of_prev_attempts      int64
studied_credits           int64
disability               object
final_result             object
id_assessment             int64
date_submitted            int64
is_banked                 int64
score                   float64
assessment_type          object
date_y                  float64
weight                  float64
dtype: object

In [7]:
# View unique values in each column
# for col in df:
#   print(df[col].unique())

code_module                   0.000000
code_presentation             0.000000
id_student                    0.000000
gender                        0.000000
region                        0.000000
highest_education             0.000000
imd_band                      6.063863
age_band                      0.000000
num_of_prev_attempts          0.000000
studied_credits               0.000000
disability                    0.000000
final_result                  0.000000
id_site                       0.004021
date_x                        0.004021
sum_click                     0.004021
id_assessment                 0.121118
date_submitted                0.121118
is_banked                     0.121118
score                         0.164842
assessment_type               0.121118
date_y                        1.883209
weight                        0.121118
date_registration             0.001936
module_presentation_length    0.000000
activity_type                 0.004021
dtype: float64

In [10]:
allCol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10744851 entries, 0 to 10744850
Data columns (total 25 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   code_module                 object 
 1   code_presentation           object 
 2   id_student                  int64  
 3   gender                      object 
 4   region                      object 
 5   highest_education           object 
 6   imd_band                    object 
 7   age_band                    object 
 8   num_of_prev_attempts        int64  
 9   studied_credits             int64  
 10  disability                  object 
 11  final_result                object 
 12  id_site                     float64
 13  date_x                      float64
 14  sum_click                   float64
 15  id_assessment               float64
 16  date_submitted              float64
 17  is_banked                   float64
 18  score                       float64
 19  assessment_type    

In [8]:
#Creating variables for indexing continuous and categorical variables
conCol = ['id_student', 'id_site', 'date_x', 'sum_click', 'num_of_prev_attempts', 
          'studied_credits', 'id_assessment ', 'date_submitted', 'is_banked', 'score', 'date_y', 'weight']

catCol = ['code_module ', 'code_presentation', 'gender', 'region', 'highest_education', 
          'imd_band', 'age_band', 'disability', 'final_result', 'assessment_type' ]

TypeError: unhashable type: 'Series'

# Data Understanding 2 #
*Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.*

# Modeling and Evaluation 1 #
*Train and adjust parameters*

# Modeling and Evaluation 2	#
*Evaluate and Compare*

# Modeling and Evaluation 3 #
*Visualize Results*

# Modeling and Evaluation 4 #
*Summarize the Ramifications*

# Deployment #
*Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?*

# Exceptional Work #